In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.sql.execution.arrow.enabled":"true"
         }
}

[I 2023-03-17 12:59:07,576.576 configure_magic] Magic cell payload received: {"conf": {"spark.pyspark.python": "python3", "spark.pyspark.virtualenv.enabled": "true", "spark.pyspark.virtualenv.type": "native", "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv", "spark.sql.execution.arrow.enabled": "true"}, "proxyUser": "assumed-role_fdp_blitvin-Isengard"}

[I 2023-03-17 12:59:07,577.577 configure_magic] Sending request to update kernel. Please wait while the kernel will be refreshed.


The kernel is successfully refreshed.

In [31]:
sc.install_pypi_package("polygon")

In [2]:
#%pip install awswrangler

In [14]:
import pyEX as p
import requests
import json
import boto3
import pandas as pd
import numpy as np
#import vectorbt as vbt
import matplotlib.pyplot as plt 
import itertools 
import datetime 
import awswrangler as wr
import vectorbt as vbt
import pyspark.sql.functions as py_f
import pyspark.sql.types as py_t
from string import Template
import time
import random
isSandbox = False
if isSandbox:
    token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
    iex_ver = 'sandbox'
else:
    #token ='pk_2e94555e43da4135a6032433c6b18fa5'pk_79d147436c1349f3abbec37591323e52
    token ='pk_79d147436c1349f3abbec37591323e52'
    iex_ver = 'stable'
pyEX_cl = p.Client(api_token=token)
timeframe='max'

In [16]:
refresh_data = False
def get_ETF_constituents(symbol,etf_holdings_dates_url,etf_holdings_url,schema):
    time.sleep(random.randrange(1,60))
    if isinstance(symbol, str):
        pass
    else:
        symbol=symbol.values[0][0]
         
    cols= [i.split(" ")[0] for i in schema.split(",")]
    vals= ['' if i.split(" ")[1] == 'string' else 0  for i in schema.split(",") ]
    empty_pd= pd.DataFrame([vals],columns=cols)
    findataprep_dates_url = etf_holdings_dates_url.substitute(symbol=symbol)
    empty_pd['etf_symbol']=f"{symbol} : {findataprep_dates_url} : {etf_holdings_url}"
    try:
        dates = requests.get(findataprep_dates_url)
        all_dates=dates.json()
    except  Exception as l_exc:
        ret_pd=empty_pd
        ret_pd['name']=l_exc 
        ret_pd['title']='exception' 
        return(ret_pd)
    all_date_pd = []
    for one_d in all_dates:  
        try:
            findataprep_url = etf_holdings_url.substitute(symbol=symbol,date=one_d.get('date'))
            one_date_json = requests.get(findataprep_url).json()
            one_date_pd=pd.DataFrame(one_date_json)
            one_date_pd['etf_symbol']=symbol
        except:
            one_date_pd=empty_pd
            one_date_pd['etf_symbol']=symbol    
        all_date_pd=all_date_pd+[one_date_pd] 
    if len(all_date_pd)>0:
        ret_pd = pd.concat(all_date_pd)
    else:
        ret_pd=empty_pd
    return(ret_pd)
#@py_f.pandas_udf("date timestamp, close double, open double, symbol string", py_f.PandasUDFType.GROUPED_MAP)
#@py_f.pandas_udf("symbol string", py_f.PandasUDFType.GROUPED_MAP)
def get_new_data_df(symbol):
    local_sym=str(symbol.values[0][0])
    try:
        hwm_l=highwatermark_pd_bc.value
        columns_to_capture=columns_to_capture_bc.value
        iex_base_url=iex_base_url_bc.value
        sym_pd = hwm_l.query(f"symbol=='{local_sym}'")
        if len(sym_pd)>0:
            max_date = sym_pd['max_date'].iloc[0]
        else:
            max_date = datetime.datetime.now()-datetime.timedelta(days=365*50)
        last_days=(datetime.datetime.now()-max_date).days

        pyEX_cl = p.Client(api_token=iex_token_bc.value)
        final_url = iex_base_url.format(local_sym,iex_token_bc.value,int(last_days))
        #'https://cloud.iexapis.com/stable/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
        res = requests.get(final_url)
        ret_pd = pd.DataFrame(json.loads(res.text))#[columns_to_capture]
        ret_pd=ret_pd.query(f"date>'{max_date.strftime('%Y-%m-%d')}'")
        ret_pd['date']=pd.to_datetime(ret_pd['date'])
        print('panda UDF')
        #return(ret_pd[columns_to_capture].query(f"date>'{max_date}'"))
    except:
        ret_pd=pd.DataFrame([{'date': datetime.datetime.now()-datetime.timedelta(days=365*50),
                              'close': None, 
                              'open': None, 
                              'symbol': local_sym}])
    #return(ret_pd[columns_to_capture],max_date,hwm_l,local_sym,sym_pd)
    return(ret_pd[columns_to_capture])
    #return(pd.DataFrame([res.text],columns=['symbol']))
    
class MarketDataGateway():
    def __init__(self,isSandbox=False):
        self.gold_etf_symbols = ['GLD','IAU','GLDM','SGOL','IAUM','BAR','OUNZ','AAAU','UGL','DBP','FGLD',]
        self.silver_etf_symbols = ['SLV','SIVR','AGQ']
        self.oil_etf_symbols = ['USO','UCO','DBO','SCO']
        self.natgas_etf_symbols = ['BOIL','UNG','KOLD']
        self.agri_etf_symbols = ['DBA','FAAR','TILL']
        self.commod_divers_etf_symbols = ['PDBC','FTGC','DBC','GSG','BCI','DJP','CMDY','COM','BCD','USCI','GCC','COMB','DJCB','BDRY',]
        self.fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
        self.bitcoin_etf_symbols = ['BITO','BTF','XBTF','BITS','GBTC','BITW','BLOK','BLCN','LEGR','SPBC','BITQ','BKCH','DAPP','BTCFX']
        self.bitcoin_equity_proxy_symbols = ['COIN','MSTR']
        self.crypto_mining_symbols = ['RIOT','CAN','HUT','HIVE','MARA','BTCM','BTBT','BITF']
        self.commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
        self.fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
        self.equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
                'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                      'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
                    'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                      'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
        self.sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
        self.bench_symbol=['SPY','QQQ','IVV']
        self.all_symbols =self.fx_etf_symbols+\
                            self.bitcoin_etf_symbols+\
                            self.commod_etf_symbols+\
                            self.bitcoin_equity_proxy_symbols+\
                            self.crypto_mining_symbols+\
                            self.fi_etf_symbols+\
                            self.equity_etf_symbols+\
                            self.bench_symbol+\
                            self.sectors+\
                            self.gold_etf_symbols+\
                            self.silver_etf_symbols+\
                            self.oil_etf_symbols+\
                            self.natgas_etf_symbols+\
                            self.agri_etf_symbols+\
                            self.commod_divers_etf_symbols
        self.all_symbols_df = spark.createDataFrame(pd.DataFrame(self.all_symbols,columns=['symbol']))
        #all_symbols
        self.columns_to_capture = ['date','close','open','symbol']
        self.secrets = {
            'IEX':{},
            'alpaca':{ 'token':'alpaca_marketdata_key',
                      'secret':'alpaca_marketdata_secret'}
                       }
        self.data_folder = 's3://fsidatalake/eod_market_data/etf_trend_following/'
        self.parquet_file_out =f"{self.data_folder}/etf_market_data.parquet"
        if isSandbox:
            self.iex_token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
            self.iex_ver = 'sandbox'
        else:
            #self.iex_token ='pk_2e94555e43da4135a6032433c6b18fa5' pk_79d147436c1349f3abbec37591323e52
            self.iex_token ='pk_79d147436c1349f3abbec37591323e52' 
            self.iex_ver = 'stable'
        self.iex_base_url = 'https://cloud.iexapis.com/'+self.iex_ver+'/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
        
        self.findataprep_config={}
        self.findataprep_config['etf_holdings_dir']='b753de15443e37f6598e7702811dfad1'
        self.findataprep_config['apikey']='b753de15443e37f6598e7702811dfad1'
        self.findataprep_config['etf_holdings_dates_url']=Template(
            f"https://financialmodelingprep.com/api/v4/etf-holdings/portfolio-date?symbol=$symbol&apikey={self.findataprep_config['apikey']}")
        self.findataprep_config['etf_holdings_url']=Template(
            f"https://financialmodelingprep.com/api/v4/etf-holdings?symbol=$symbol&date=$date&apikey={self.findataprep_config['apikey']}")
        self.findataprep_config['etf_holdings_columns'] = ['cik', 'acceptanceTime', 'date', 'symbol', 'name', 'lei', 'title',
                                                           'cusip', 'isin', 'balance', 'units', 'cur_cd', 'valUsd', 'pctVal',
                                                           'payoffProfile', 'assetCat', 'issuerCat', 'invCountry',
                                                           'isRestrictedSec', 'fairValLevel', 'isCashCollateral',
                                                           'isNonCashCollateral', 'isLoanByFund', 'etf_symbol']
        self.findataprep_config['etf_holdings_column_types']= ['string','string','string',
                                                                'string','string','string','string','string','string',
                                                                'int','string','string','int','float',
                                                                'string','string','string','string','string',
                                                                'string','string','string','string','string',]
        self.findataprep_config['etf_holdings_udf_schema']=",".join([f"{i[0]} {i[1]}" for i in zip( self.findataprep_config['etf_holdings_columns'], self.findataprep_config['etf_holdings_column_types'])])
        self.findataprep_config['etf_holdings_output_file']="s3://fsidatalake/ETF_constituents/etf_holdings.parquet"
        
        
    def set_highwatermarks(self):
        try:
            self.old_df = spark.read.parquet(self.parquet_file_out)
        except:
            self.old_df = spark.createDataFrame(pd.DataFrame([{'symbol':'----','date':datetime.datetime.now()-datetime.timedelta(days=365*50)}]))
        self.highwatermark_pd = self.old_df.groupby('symbol').agg(py_f.max("date").alias("max_date"),
                             py_f.min("date").alias("min_date"),
                             (py_f.max("date")-py_f.min("date")).alias("date_length")
                            ).toPandas()
    def set_new_symbol_df(self):
        self.new_sym_df=self.all_symbols_df.groupby("symbol")\
                        .applyInPandas(get_new_data_df, schema="date timestamp, close double, open double, symbol string")\
                        .where('close is not null')
    def write_new_symbol_df(self):
        self.write_stats=self.new_sym_df.groupby('symbol').agg(py_f.count('symbol').alias('data_count'))
        self.new_sym_df.write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("append")\
            .parquet(self.parquet_file_out)
    def rewrite_if_dups(self):
        self.old_df_after_write = spark.read.parquet(self.parquet_file_out)
        self.dup_df = self.old_df_after_write.distinct().groupby('date','symbol').count()
        dup_count=self.dup_df.where('count>1').count()
        if dup_count>0:        
            print(f"found:{dup_count} dups; sample:{self.dup_df.show()}")
            self.old_df_after_write.groupby('date','symbol').agg(py_f.mean('close').alias('close'),py_f.mean('open').alias('open'))\
            .write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("overwrite")\
            .parquet(self.parquet_file_out.replace('.parquet','_temp.parquet'))
        else:
            print(f"no dups")
            
    def get_secret(self,secret_name,region_name):
        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            # For a list of exceptions thrown, see
            # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
            raise e

        # Decrypts secret using the associated KMS key.
        secret = json.loads(get_secret_value_response['SecretString'])[secret_name]
        return(secret)
        

mdg=MarketDataGateway()
if refresh_data:
    mdg.set_highwatermarks()
    # set bc vars
    highwatermark_pd_bc =sc.broadcast(mdg.highwatermark_pd)
    columns_to_capture_bc =sc.broadcast(mdg.columns_to_capture)
    iex_token_bc =sc.broadcast(mdg.iex_token)
    iex_base_url_bc =sc.broadcast(mdg.iex_base_url)
    # set bc vars
    mdg.set_new_symbol_df()
    mdg.write_new_symbol_df()
    print(mdg.write_stats.toPandas().sort_values(['symbol']))
    #mdg.rewrite_if_dups()



In [17]:
mdg.findataprep_config['etf_holdings_output_file']

's3://fsidatalake/ETF_constituents/etf_holdings.parquet'

In [18]:
random.randrange(1, 30)

16

In [19]:
etf_holdings_dates_url,etf_holdings_url=mdg.findataprep_config['etf_holdings_dates_url'],mdg.findataprep_config['etf_holdings_url']
schema=mdg.findataprep_config['etf_holdings_udf_schema']
etf_const_df=mdg.all_symbols_df.where(py_f.col('symbol').isin(mdg.all_symbols)).groupby('symbol')\
                .applyInPandas(lambda x: get_ETF_constituents(x,etf_holdings_dates_url,etf_holdings_url,schema), schema=schema )
if True:
    etf_const_df.coalesce(1).write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("overwrite")\
            .parquet(mdg.findataprep_config['etf_holdings_output_file'])

Py4JJavaError: An error occurred while calling o913.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:279)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:193)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:103)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:107)
	at org.a

In [20]:
if False:
    x=mdg.bench_symbol[2]
    get_ETF_constituents(x,etf_holdings_dates_url,etf_holdings_url)
else:
    etf_const_df.where('title="exception"').show(truncate=False)

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "pyarrow/array.pxi", line 1044, in pyarrow.lib.Array.from_pandas
  File "pyarrow/array.pxi", line 316, in pyarrow.lib.array
  File "pyarrow/array.pxi", line 83, in pyarrow.lib._ndarray_to_array
  File "pyarrow/error.pxi", line 123, in pyarrow.lib.check_status
pyarrow.lib.ArrowTypeError: Expected bytes, got a 'JSONDecodeError' object


In [22]:
import requests
from string import Template

url = Template("https://eodhistoricaldata.com/api/fundamentals/$symbol?api_token=64143bca135077.45835807")
ix_symbols = [
 '5QQ0.F'
,'SPI20.INDX'
,'0GZF.XETRA'
,'N225.INDX'
,'0GZG.XETRA'
,'3E3M.XETRA'
,'SSMI.INDX'
,'VIX.INDX'
,'EGV2.XETRA'
,'GSPTSE.INDX'
,'0GZI.XETRA'
,'CMCI.LSE'
,'FCHI.INDX'
,'GSPC.INDX'
,'NDX.INDX'
,'GDAXI.INDX'
,'0GZH.XETRA'
,'FTSE.INDX'
,'3E3S.XETRA']

res_dict={}
res_dict['Components']=[]
res_dict['HistoricalTickerComponents']=[]
for symbol in ix_symbols:
    final_url=url.substitute(symbol=symbol)
    print(final_url)
    r = requests.get(final_url)
    res_json= json.loads(r.text)
    print( res_json.keys())
    for one_key in [i for i in res_json.keys() if 'Components' in i]:

        print(one_key)
        ix_const=pd.DataFrame(res_json.get(one_key)).transpose()
        ix_const['index_symbol']=symbol
        ix_const['index_name']=res_json.get('General').get('Name')
        res_dict[one_key]=res_dict[one_key]+[ix_const]
    #ix_const
current_const_df = spark.createDataFrame(pd.concat(res_dict['Components']))


https://eodhistoricaldata.com/api/fundamentals/5QQ0.F?api_token=64143bca135077.45835807
dict_keys(['General', 'Highlights', 'Valuation', 'SharesStats', 'Technicals', 'SplitsDividends', 'Holders', 'InsiderTransactions', 'outstandingShares', 'Earnings', 'Financials'])
https://eodhistoricaldata.com/api/fundamentals/SPI20.INDX?api_token=64143bca135077.45835807
dict_keys(['General', 'Components'])
Components
https://eodhistoricaldata.com/api/fundamentals/0GZF.XETRA?api_token=64143bca135077.45835807
dict_keys(['General', 'Components'])
Components
https://eodhistoricaldata.com/api/fundamentals/N225.INDX?api_token=64143bca135077.45835807
dict_keys(['General', 'Components'])
Components
https://eodhistoricaldata.com/api/fundamentals/0GZG.XETRA?api_token=64143bca135077.45835807
dict_keys(['General', 'Components'])
Components
https://eodhistoricaldata.com/api/fundamentals/3E3M.XETRA?api_token=64143bca135077.45835807
dict_keys(['General', 'Components'])
Components
https://eodhistoricaldata.com/api/

In [23]:

current_const_df = spark.createDataFrame(pd.concat(res_dict['Components']))
current_const_df.coalesce(1).write\
            .option("header",True)\
            .partitionBy('index_symbol')\
            .mode("overwrite")\
            .parquet('s3://fsidatalake/ETF_constituents/index_const.parquet')


In [24]:
current_const_df_hist = spark.createDataFrame(pd.concat(res_dict['HistoricalTickerComponents']))
current_const_df_hist.coalesce(1).write\
            .option("header",True)\
            .partitionBy('index_symbol')\
            .mode("overwrite")\
            .parquet('s3://fsidatalake/ETF_constituents/index_const_hist.parquet')

In [26]:
def polygon_tickers():
    final_url="https://api.polygon.io/v3/reference/tickers?active=true&apiKey=ylNi3UUvnCb2VzFfIz1gpJwX5cvzbNdF"
    r = requests.get(final_url)
    res_json= json.loads(r.text)
    return(r)
resp=polygon_tickers()  

In [30]:
json.loads(resp.text)

{'results': [{'ticker': 'A',
   'name': 'Agilent Technologies Inc.',
   'market': 'stocks',
   'locale': 'us',
   'primary_exchange': 'XNYS',
   'type': 'CS',
   'active': True,
   'currency_name': 'usd',
   'cik': '0001090872',
   'composite_figi': 'BBG000C2V3D6',
   'share_class_figi': 'BBG001SCTQY4',
   'last_updated_utc': '2023-03-16T00:00:00Z'},
  {'ticker': 'AA',
   'name': 'Alcoa Corporation',
   'market': 'stocks',
   'locale': 'us',
   'primary_exchange': 'XNYS',
   'type': 'CS',
   'active': True,
   'currency_name': 'usd',
   'cik': '0001675149',
   'composite_figi': 'BBG00B3T3HD3',
   'share_class_figi': 'BBG00B3T3HF1',
   'last_updated_utc': '2023-03-16T00:00:00Z'},
  {'ticker': 'AAA',
   'name': 'AXS First Priority CLO Bond ETF',
   'market': 'stocks',
   'locale': 'us',
   'primary_exchange': 'ARCX',
   'type': 'ETF',
   'active': True,
   'currency_name': 'usd',
   'cik': '0001776878',
   'composite_figi': 'BBG01B0JRCS6',
   'share_class_figi': 'BBG01B0JRCT5',
   'last_

In [ ]:


from polygon import ReferenceClient


KEY = 'ylNi3UUvnCb2VzFfIz1gpJwX5cvzbNdF'

reference_client = ReferenceClient(KEY)

# getting ALL ticker names from polygon

responses = []  # just creating a list to store all responses that we get. You can use your own approach here

response = reference_client.get_tickers(limit=1000)

while 'next_url' in response.keys():
    next_page = reference_client.get_next_page(response)
    responses.append(next_page)
    
print(f'All pages received. Total pages: {len(responses)}')